In [ ]:
import pandas as pd #módulo para manipular base de dados
import numpy as np #módulo para manipular os vetores
from matplotlib import pyplot as plt #biblioteca para plotar os gráficos
from sklearn.cluster import KMeans #biblioteca para utilizar o KMeans (algorítmo de Clusterização)
import plotly.graph_objs as go, plotly.express as px #Módulos para edição de layouts dos gráficos
import seaborn as sns

import plotly.figure_factory as ff
import numpy as np

In [ ]:
#Chamando a base de dados - AJUSTE O DIRETÓRIO CONFORME SUA ESTAÇÃO DE TRABALHO
df_ml = pd.read_csv(r"C:\Users\bruno\Desktop\Machine Learning Training\ALTURA_POR_PAIS\Height of Male and Female by Country 2022.csv")
df_ml

In [ ]:
#Visualizando as propriedades e atributos do DF
df_ml.info()

In [ ]:
#Setando o índice
df_ml.set_index('Country Name', inplace=True)
df_ml.info()

In [ ]:
#Definindo as variáveis (métricas) que serão consideradas na clusterização
df_ml = df_ml.iloc[:,[1,2]]
df_ml

In [ ]:
#Removendo o título do índice
df_ml.index.name = None
df_ml

In [ ]:
#Renomeando (e traduzindo) colunas para evitar problemas no modelo
df_ml = df_ml.rename(columns={'Male Height in Cm' : 'Altura_Homens_Cm', 'Female Height in Cm' : 'Altura_Mulheres_Cm'})
df_ml

In [ ]:
#Análise inicial de dispersão dos dados
plt.scatter(
    x=df_ml.Altura_Mulheres_Cm, 
    y=df_ml.Altura_Homens_Cm,
    s = 15, 
    c = '#481F67')
plt.xlabel('Altura das Mulheres em centímetros')
plt.ylabel('Altura dos Homens em centímetros')
plt.title('Análise inicial de dispersão')
plt.show()

In [ ]:
#Criando objeto para armazenar as variáveis
x = df_ml.iloc[:,[0,1]].values

In [ ]:
#Executando o método Elbow
elbow_ml = df_ml.iloc[:,[0,1]]

In [ ]:
#Criando  Elbow method (Método Elbow, ou Curva de Cotovelo) para encontrar o número apropriado de Clusters
wcss = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(elbow_ml)
    wcss.append(kmeans.inertia_)
    
# Mostra o Gráfico
plt.plot(range(1, 11), 
         wcss,
         'r--',
         c = '#D13B56')
plt.grid(True)
ax = plt.axes() 
ax.set_facecolor("#F5F5F5") 
plt.title('Curva de Cotovelo')
plt.xlabel('Numero de Clusters')
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

In [ ]:
#Se não colocarmos o número de clusters, o valor default de número de clusters é de 8 clusters.
modelo_cluster = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)

#Cria o modelo de clusterização para agrupar os dados
modelo_cluster = modelo_cluster.fit(x)

#Mostra quais são as coordenadas de cada centroid para cada dimensão. Temos 2 dimensões e 3 clusters.
modelo_cluster.cluster_centers_

In [ ]:
# Mostra a quantidade de clusters
modelo_cluster.n_clusters

In [ ]:
#Criando nova coluna no DataFrame com os grupos
df_ml['Agrupamento'] = modelo_cluster.labels_
df_ml

In [ ]:
#Mostra os labels
labels = modelo_cluster.labels_
labels

In [ ]:
#Renomeando os grupos
df_ml['Agrupamento'] = df_ml['Agrupamento'].map({0:'Grupo 1',
                             1:'Grupo 2',
                             2:'Grupo 3'})
df_ml

In [ ]:
#Faz a clusterização dos dados usando o modelo criado
grupos = modelo_cluster.predict(x)

#Mostra o Gráfico por Clusters
sns.scatterplot(x=df_ml.Altura_Mulheres_Cm, y=df_ml.Altura_Homens_Cm, hue=df_ml.Agrupamento)
plt.show()

In [ ]:
#Visualiza os clusters com os Centroides
plt.scatter(x[grupos == 0, 0], x[grupos == 0, 1], s = 100, c = '#A6A6A6') #label = 'Grupo 1'
plt.scatter(x[grupos == 1, 0], x[grupos == 1, 1], s = 100, c = '#066664') #label = 'Grupo 2'
plt.scatter(x[grupos == 2, 0], x[grupos == 2, 1], s = 100, c = '#2CF4EF') #label = 'Grupo 3'
plt.scatter(modelo_cluster.cluster_centers_[:, 0], 
            modelo_cluster.cluster_centers_[:,1], 
            s = 150, 
            c = '#F56161', 
            label = 'Centroides')
plt.style.use('classic')
plt.title('Clustering', fontsize = 20)
plt.xlabel('Altura dos Homens em centímetros')
plt.ylabel('Altura dos Mulheres em centímetros')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Visualização adicional
sns.pairplot(df_ml, hue="Agrupamento")
plt.style.use('classic')
plt.show()

In [ ]:
#CASO QUEIRA EXPORTAR O NOVO DataFrame PARA O EXCEL, BASTA AJUSTAR O DIRETÓRIO ABAIXO EM QUE DESEJA GUARDAR O ARQUIVO
df_ml.to_excel(r'C:\Users\bruno\Desktop\Machine Learning Training\ALTURA_POR_PAIS\df_grupos.xlsx', index = True)